In [2]:
import dask.dataframe as dd
from dask.distributed import Client
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import pyarrow.compute as pc
import gc
from decimal import Decimal  # Add this import statement



In [3]:
import gc

# Function to flush the cache
def flush_cache():
    gc.collect()

In [13]:
flush_cache()
speriod=50000
samples=5
# Define the folder containing the Parquet files
folder_path = r'D:\RISHIN\13_ILC_TASK1\input\PARQUET_FILES'

# List all Parquet files in the folder
parquet_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.parquet')]
output_folder_path = "D:\RISHIN\TESTING FOLDER\ILC_TEST_3"

# Check if there are any Parquet files in the folder
if parquet_files:
    # Read the first Parquet file in chunks
    parquet_file = pq.ParquetFile(parquet_files[0])
    for batch in parquet_file.iter_batches(batch_size=1000):
        # Convert the first batch to a PyArrow Table
        table = pa.Table.from_batches([batch])
        
        # Convert the PyArrow Table to a Pandas DataFrame
        df = table.to_pandas()
        
        # Extract the first value of LocationName and split it by '_'
        location_name = df['LocationName'].iloc[0]
        country = location_name.split('_')[0]
        
        
        # Define the main folder path
        main_folder_path = os.path.join(output_folder_path, f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_Losses')
        
        # Define subfolders
        subfolders = ['EP', 'PLT', 'STATS']
        nested_folders = ['Lob', 'Portfolio']
        innermost_folders = ['GR', 'GU']
        
        # Create the main folder and subfolders
        for subfolder in subfolders:
            subfolder_path = os.path.join(main_folder_path, subfolder)
            os.makedirs(subfolder_path, exist_ok=True)
            
            for nested_folder in nested_folders:
                nested_folder_path = os.path.join(subfolder_path, nested_folder)
                os.makedirs(nested_folder_path, exist_ok=True)
                
                for innermost_folder in innermost_folders:
                    innermost_folder_path = os.path.join(nested_folder_path, innermost_folder)
                    os.makedirs(innermost_folder_path, exist_ok=True)
        
        print(f"Folders created successfully at {main_folder_path}")
        break  # Process only the first batch
else:
    print("No Parquet files found in the specified folder.")

Folders created successfully at D:\RISHIN\TESTING FOLDER\ILC_TEST_3\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses


In [11]:
#for LOB AUTO portfolio

In [ ]:
'LobName','LobId','EventId', 'PeriodId','Weight', 'EventDate','LossDate','Region','Peril'

In [ ]:
# import pyarrow as pa
# import pyarrow.parquet as pq
# import pyarrow.compute as pc
# import pyarrow.dataset as ds
# import os

# processing_folder_path = os.path.join(output_folder_path, 'processing')
# os.makedirs(processing_folder_path, exist_ok=True)

# # Initialize an empty list to store the results
# final_grouped_table_1 = []

# # Define the chunk size
# chunk_size = 4_000_000  # 4 million rows per chunk

# # Process each Parquet file individually
# for file in parquet_files:
#     # Create a dataset from the Parquet file
#     dataset = ds.dataset(file, format="parquet")

#     # Read the dataset in chunks
#     for batch in dataset.to_batches(batch_size=chunk_size):
#         table = pa.Table.from_batches([batch])
        
#         # Filter the table
#         table = table.filter(pc.equal(table['LobName'], 'AUTO'))

#         # Perform the aggregation: sum the Loss column grouped by EventId, PeriodId, and EventDate
#         grouped_table_1 = table.group_by(['EventId', 'PeriodId', 'EventDate']).aggregate([('Loss', 'sum')])
#         grouped_table_1 = grouped_table_1.rename_columns(['EventId', 'PeriodId', 'EventDate', 'Sum_Loss'])
        
#         # Write intermediate results to disk
#         pq.write_table(grouped_table_1, os.path.join(processing_folder_path, f'grouped_table_1_{os.path.basename(file)}'))

# # Read all intermediate files and concatenate them
# intermediate_files_1 = [os.path.join(processing_folder_path, f) for f in os.listdir(processing_folder_path) if f.startswith('grouped_table_1_')]

# final_grouped_table_1 = [pq.read_table(f) for f in intermediate_files_1]

# final_table_1 = pa.concat_tables(final_grouped_table_1)

# # Perform final grouping and sorting
# f_grouped_table_1 = final_table_1.group_by(['EventId', 'PeriodId', 'EventDate']).aggregate([('Sum_Loss', 'sum')])
# sorted_final_table_1 = f_grouped_table_1.sort_by([('Sum_Loss_sum', 'descending')])

# # Convert to pandas DataFrame
# dataframe_1_oep = sorted_final_table_1.to_pandas()

# # Delete all non-concatenated files
# for f in intermediate_files_1:
#     os.remove(f)

# print(dataframe_1_oep)

In [14]:
processing_folder_path = os.path.join(output_folder_path, 'processing')
os.makedirs(processing_folder_path, exist_ok=True)

# Initialize an empty list to store the results
final_grouped_table_1 = []

# Process each Parquet file individually
for file in parquet_files:
    # Read the Parquet file into a PyArrow Table
    table = pq.read_table(file)
    
    # Perform the aggregation: max the Loss column grouped by EventId, PeriodId, LobName, and LocationId
    table = table.filter(pc.equal(table['LobName'], 'AUTO'))

    grouped_table_1 = table.group_by(['EventId', 'PeriodId', 'EventDate']).aggregate([('Loss', 'sum')])
    grouped_table_1 = grouped_table_1.rename_columns(['EventId', 'PeriodId', 'EventDate','Sum_Loss'])
   
    # Write intermediate results to disk
    pq.write_table(grouped_table_1, os.path.join(processing_folder_path, f'grouped_table_1_{os.path.basename(file)}'))

# Read all intermediate files and concatenate them
intermediate_files_1 = [os.path.join(processing_folder_path, f) for f in os.listdir(processing_folder_path) if f.startswith('grouped_table_1_')]

final_grouped_table_1 = [pq.read_table(f) for f in intermediate_files_1]

final_table_1 = pa.concat_tables(final_grouped_table_1)

# Perform final grouping and sorting
f_grouped_table_1 = final_table_1.group_by(['EventId', 'PeriodId', 'EventDate']).aggregate([('Sum_Loss', 'sum')])
sorted_final_table_1 = f_grouped_table_1.sort_by([('Sum_Loss_sum', 'descending')])

# Convert to pandas DataFrames
dataframe_1_oep = sorted_final_table_1.to_pandas()



# Delete all non-concatenated files
for f in intermediate_files_1 :
    os.remove(f)





In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import os

processing_folder_path = os.path.join(output_folder_path, 'processing')
os.makedirs(processing_folder_path, exist_ok=True)

# Initialize an empty list to store the results
final_grouped_table_1 = []

# Process each Parquet file individually
for file in parquet_files:
    # Read the Parquet file into a PyArrow Table
    table = pq.read_table(file)
    
    # Perform the aggregation: max the Loss column grouped by EventId, PeriodId, LobName, and LocationId
    table = table.filter(pc.equal(table['LobName'], 'AUTO'))

    grouped_table_1 = table.group_by(['EventId', 'PeriodId', 'EventDate']).aggregate([('Loss', 'sum')])
    grouped_table_1 = grouped_table_1.rename_columns(['EventId', 'PeriodId', 'EventDate', 'Sum_Loss'])
   
    # Write intermediate results to disk
    pq.write_table(grouped_table_1, os.path.join(processing_folder_path, f'grouped_table_1_{os.path.basename(file)}'))

# Read all intermediate files and concatenate them
intermediate_files_1 = [os.path.join(processing_folder_path, f) for f in os.listdir(processing_folder_path) if f.startswith('grouped_table_1_')]

final_grouped_table_1 = [pq.read_table(f) for f in intermediate_files_1]

final_table_1 = pa.concat_tables(final_grouped_table_1)

# Perform final grouping and sorting
f_grouped_table_1 = final_table_1.group_by(['EventId', 'PeriodId', 'EventDate']).aggregate([('Sum_Loss', 'sum')])
sorted_final_table_1 = f_grouped_table_1.sort_by([('Sum_Loss_sum', 'descending')])

# Convert to pandas DataFrame
dataframe_1_oep = sorted_final_table_1.to_pandas()

# Delete all non-concatenated files
for f in intermediate_files_1:
    os.remove(f)

print(dataframe_1_oep)

In [ ]:
dataframe_1_aep

,EventId,PeriodId,LobName,Sum_Loss_sum
0,53895970,207557,SPER,2.250859e+10
1,53891598,149148,SPER,2.197889e+10
2,53893301,4312,SPER,2.160561e+10
3,53893301,154312,SPER,2.092983e+10
4,53891598,199148,SPER,1.993543e+10
...,...,...,...,...
7374263,53876702,165828,AUTO,1.870107e-03
7374264,53865125,130081,AGR,1.669423e-03
7374265,53881107,82560,AUTO,8.474226e-04
7374266,53872002,212031,AGR,6.278728e-04


In [52]:
dataframe_1_oep

,EventId,PeriodId,EventDate,Sum_Loss_sum
0,53891598,149148,2020-05-13,1.654085e+09
1,53891598,199148,2020-05-13,1.509087e+09
2,53895970,207557,2020-05-30,1.359457e+09
3,53893301,4312,2020-11-10,1.319517e+09
4,53893301,154312,2020-11-10,1.253157e+09
...,...,...,...,...
1550192,53856304,53560,2020-03-26,3.063393e-03
1550193,53855161,89818,2020-02-09,2.959355e-03
1550194,53867896,132912,2020-02-29,2.819001e-03
1550195,53876702,165828,2020-10-21,1.870107e-03


In [61]:
dataframe_2 = dataframe_1_oep.groupby(['PeriodId'], as_index=False).agg({'Sum_Loss_sum': 'max'})
dataframe_2.rename(columns={'Sum_Loss_sum': 'Max_Loss'}, inplace=True)
dataframe_2 = dataframe_2.sort_values(by='Max_Loss', ascending=False).reset_index(drop=True)

dataframe_2['rate'] = (1 / (speriod * samples))
dataframe_2['cumrate'] = dataframe_2['rate'].cumsum().round(6)
dataframe_2['RPs'] = (1 / dataframe_2['cumrate'])
dataframe_2['TCE_OEP_1'] = ((dataframe_2['Max_Loss'] - dataframe_2['Max_Loss'].shift(-1)) * (dataframe_2['cumrate'] + dataframe_2['cumrate'].shift(-1)) * 0.5)
dataframe_2['TCE_OEP_2'] = (dataframe_2['TCE_OEP_1'].shift().cumsum() * dataframe_2['RPs'])
dataframe_2['TCE_OEP_Final'] = (dataframe_2['TCE_OEP_2'] + dataframe_2['Max_Loss'])

dataframe_3 = dataframe_1_oep.groupby(['PeriodId'], as_index=False).agg({'Sum_Loss_sum': 'sum'})
dataframe_3.rename(columns={'Sum_Loss_sum': 'S_Sum_Loss'}, inplace=True)
dataframe_3 = dataframe_3.sort_values(by='S_Sum_Loss', ascending=False).reset_index(drop=True)

dataframe_3['rate'] = (1 / (speriod * samples))
dataframe_3['cumrate'] = dataframe_3['rate'].cumsum().round(6)
dataframe_3['RPs'] = (1 / dataframe_3['cumrate'])
dataframe_3['TCE_AEP_1'] = ((dataframe_3['S_Sum_Loss'] - dataframe_3['S_Sum_Loss'].shift(-1)) * (dataframe_3['cumrate'] + dataframe_3['cumrate'].shift(-1)) * 0.5)
dataframe_3['TCE_AEP_2'] = (dataframe_3['TCE_AEP_1'].shift().cumsum() * dataframe_3['RPs'])
dataframe_3['TCE_AEP_Final'] = (dataframe_3['TCE_AEP_2'] + dataframe_3['S_Sum_Loss'])






In [62]:
dataframe_2

,PeriodId,Max_Loss,rate,cumrate,RPs,TCE_OEP_1,TCE_OEP_2,TCE_OEP_Final
0,149148,1.654085e+09,0.000004,0.000004,250000.000000,869.991566,NaN,NaN
1,199148,1.509087e+09,0.000004,0.000008,125000.000000,1496.296836,1.087489e+08,1.617836e+09
2,207557,1.359457e+09,0.000004,0.000012,83333.333333,559.156488,1.971907e+08,1.556648e+09
3,4312,1.319517e+09,0.000004,0.000016,62500.000000,1194.484886,1.828403e+08,1.502357e+09
4,154312,1.253157e+09,0.000004,0.000020,50000.000000,1155.475354,2.059965e+08,1.459153e+09
...,...,...,...,...,...,...,...,...
247708,39961,9.574042e-01,0.000004,0.990836,1.009249,0.769077,9.832831e+06,9.832832e+06
247709,162941,1.812157e-01,0.000004,0.990840,1.009245,0.003417,9.832792e+06,9.832792e+06
247710,241283,1.777667e-01,0.000004,0.990844,1.009241,0.004063,9.832752e+06,9.832752e+06
247711,71005,1.736658e-01,0.000004,0.990848,1.009237,0.046673,9.832712e+06,9.832712e+06


In [57]:
dataframe_2[dataframe_2["PeriodId"]==7383]

,PeriodId,Max_Loss,rate,cumrate,RPs,TCE_OEP_1,TCE_OEP_2,TCE_OEP_Final
124999,7383,1.454844e+06,0.000004,0.5,2.0,6.946862,1.750499e+07,1.895984e+07


In [58]:
dataframe_3

,PeriodId,S_Sum_Loss,rate,cumrate,RPs,TCE_AEP_1,TCE_AEP_2,TCE_AEP_Final
0,149148,1.660418e+09,0.000004,0.000004,250000.000000,848.360420,NaN,3.320836e+09
1,199148,1.519025e+09,0.000004,0.000008,125000.000000,1590.599828,1.060451e+08,3.038050e+09
2,207557,1.359965e+09,0.000004,0.000012,83333.333333,110.572131,2.032467e+08,2.719930e+09
3,4312,1.352067e+09,0.000004,0.000016,62500.000000,1255.639742,1.593458e+08,2.704134e+09
4,154312,1.282309e+09,0.000004,0.000020,50000.000000,1378.466997,1.902586e+08,2.564618e+09
...,...,...,...,...,...,...,...,...
247708,39961,9.574042e-01,0.000004,0.990836,1.009249,0.769077,1.245118e+07,1.914808e+00
247709,162941,1.812157e-01,0.000004,0.990840,1.009245,0.003417,1.245113e+07,3.624314e-01
247710,241283,1.777667e-01,0.000004,0.990844,1.009241,0.004063,1.245108e+07,3.555334e-01
247711,71005,1.736658e-01,0.000004,0.990848,1.009237,0.046673,1.245103e+07,3.473316e-01


In [31]:
dataframe_2.to_csv(r"D:\RISHIN\Rough\Calculated_OEP.csv")

In [30]:
dataframe_2.to_csv(r"D:\RISHIN\TESTING FOLDER\ILC_TEST_3\LOB_AUTO_SECOND.csv", index=False)

In [63]:
rps_values = [10000, 5000, 1000, 500, 250, 200, 100, 50, 25, 10, 5, 2]
fdataframe_2 = pd.DataFrame()
fdataframe_3 = pd.DataFrame()


for value in rps_values:
        closest_index_2 = (dataframe_2['RPs'] - value).abs().idxmin()
        fdataframe_2 = pd.concat([fdataframe_2, dataframe_2.loc[[closest_index_2]]])
        fdataframe_3 = pd.concat([fdataframe_3, dataframe_3.loc[[closest_index_2]]])


fdataframe_2.rename(columns={'Max_Loss': 'OEP', 'TCE_OEP_Final': 'TCE-OEP'}, inplace=True)
columns_to_keep_2 = ['RPs']
columns_to_melt_2 = ['OEP', 'TCE-OEP']
melted_df_2 = fdataframe_2.melt(id_vars=columns_to_keep_2, value_vars=columns_to_melt_2,var_name='EPType', value_name='Loss')
melted_df_2.rename(columns={'RPs': 'ReturnPeriod'}, inplace=True)
final_df_2 = melted_df_2[['EPType', 'Loss', 'ReturnPeriod']]

fdataframe_3.rename(columns={'S_Sum_Loss': 'AEP', 'TCE_AEP_Final': 'TCE-AEP'}, inplace=True)
columns_to_keep_3 = ['RPs']
columns_to_melt_3 = ['AEP', 'TCE-AEP']
melted_df_3 = fdataframe_3.melt(id_vars=columns_to_keep_3, value_vars=columns_to_melt_3,var_name='EPType', value_name='Loss')
melted_df_3.rename(columns={'RPs': 'ReturnPeriod'}, inplace=True)
final_df_3 = melted_df_3[['EPType', 'Loss', 'ReturnPeriod']]


final_df_EP_LOB_GU = pd.concat([final_df_2, final_df_3], ignore_index=True)
new_ep_type_order = ["OEP", "AEP", "TCE-OEP", "TCE-AEP"]
final_df_EP_LOB_GU['EPType'] = pd.Categorical(final_df_EP_LOB_GU['EPType'], categories=new_ep_type_order, ordered=True)
final_df_EP_LOB_GU = final_df_EP_LOB_GU.sort_values(by=['EPType', 'ReturnPeriod'], ascending=[True, False]).reset_index(drop=True)

# Define the schema to match the required Parquet file schema
schema = pa.schema([
    pa.field('EPType', pa.string(), nullable=True),
    pa.field('Loss', pa.float64(), nullable=True),
    pa.field('ReturnPeriod', pa.float64(), nullable=True),
       
])
parquet_file_path=r"D:\RISHIN\Rough\POrtfolio_AUTO.parquet"
# Convert DataFrame to Arrow Table with the specified schema
table = pa.Table.from_pandas(final_df_EP_LOB_GU, schema=schema)

# Save to Parquet
pq.write_table(table, parquet_file_path)

print(f"Parquet file saved successfully at {parquet_file_path}")

Parquet file saved successfully at D:\RISHIN\Rough\POrtfolio_AUTO.parquet


In [48]:
final_df_EP_LOB_GU.to_csv(r"D:\RISHIN\Rough\POrtfolio_AUTO.csv", index=False)

In [64]:
final_df_EP_LOB_GU

,EPType,Loss,ReturnPeriod
0,OEP,8.639885e+08,10000.0
1,OEP,6.749918e+08,5000.0
2,OEP,3.778001e+08,1000.0
3,OEP,2.847455e+08,500.0
4,OEP,2.064693e+08,250.0
5,OEP,1.867783e+08,200.0
6,OEP,1.331685e+08,100.0
7,OEP,8.957940e+07,50.0
8,OEP,5.637969e+07,25.0
9,OEP,2.190680e+07,10.0


In [67]:
dataframe_1_oep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1550197 entries, 0 to 1550196
Data columns (total 4 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   EventId       1550197 non-null  int64         
 1   PeriodId      1550197 non-null  int64         
 2   EventDate     1550197 non-null  datetime64[ns]
 3   Sum_Loss_sum  1550197 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 47.3 MB


In [62]:
#alternate 
import decimal
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

def process_and_save_parquet_2(dataframe_1_oep , parquet_file_path, speriod, samples):
    dataframe_2 = dataframe_1_oep.groupby(['PeriodId'], as_index=False).agg({'Sum_Loss_sum': 'max'})
    dataframe_2.rename(columns={'Sum_Loss_sum': 'Max_Loss'}, inplace=True)
    dataframe_2 = dataframe_2.sort_values(by='Max_Loss', ascending=False)

    dataframe_3 = dataframe_1_oep.groupby(['PeriodId'], as_index=False).agg({'Sum_Loss_sum': 'sum'})
    dataframe_3.rename(columns={'Sum_Loss_sum': 'S_Sum_Loss'}, inplace=True)
    dataframe_3 = dataframe_3.sort_values(by='S_Sum_Loss', ascending=False)

    dataframe_2['rate'] = (1 / (speriod * samples))
    dataframe_2['cumrate'] = dataframe_2['rate'].cumsum().round(6)
    dataframe_2['RPs'] = (1 / dataframe_2['cumrate'])
    dataframe_2['TCE_OEP_1'] = ((dataframe_2['Max_Loss'] - dataframe_2['Max_Loss'].shift(-1)) * 
                              (dataframe_2['cumrate'] + dataframe_2['cumrate'].shift(-1)) * 0.5)
    dataframe_2['TCE_OEP_2'] = (dataframe_2['TCE_OEP_1'].shift().cumsum() * dataframe_2['RPs'])
    dataframe_2['TCE_OEP_Final'] = (dataframe_2['TCE_OEP_2'] + dataframe_2['Max_Loss'])

    dataframe_3['rate'] = (1 / (speriod * samples))
    dataframe_3['cumrate'] = dataframe_3['rate'].cumsum()
    dataframe_3['RPs'] = (1 / dataframe_3['cumrate'])
    dataframe_3['TCE_AEP_1'] = ((dataframe_3['S_Sum_Loss'] - dataframe_3['S_Sum_Loss'].shift(-1)) * 
                              (dataframe_3['cumrate'] + dataframe_3['cumrate'].shift(-1)) * 0.5)
    dataframe_3['TCE_AEP_2'] = (dataframe_3['TCE_AEP_1'].shift().cumsum() * dataframe_3['RPs'])
    dataframe_3['TCE_AEP_Final'] = (dataframe_3['TCE_AEP_2'] + dataframe_3['S_Sum_Loss'])

    rps_values = [10000, 5000, 1000, 500, 250, 200, 100, 50, 25, 10, 5, 2]
    fdataframe_2 = pd.DataFrame()
    fdataframe_3 = pd.DataFrame()

    for value in rps_values:
        closest_index_2 = (dataframe_2['RPs'] - value).abs().idxmin()
        closest_index_3 = (dataframe_3['RPs'] - value).abs().idxmin()
        fdataframe_2 = pd.concat([fdataframe_2, dataframe_2.loc[[closest_index_2]]])
        fdataframe_3 = pd.concat([fdataframe_3, dataframe_3.loc[[closest_index_3]]])

    fdataframe_3.rename(columns={'S_Sum_Loss': 'AEP', 'TCE_AEP_Final': 'TCE-AEP'}, inplace=True)
    fdataframe_2.rename(columns={'Max_Loss': 'OEP', 'TCE_OEP_Final': 'TCE-OEP'}, inplace=True)

    

    columns_to_keep_3 = ['RPs']
    columns_to_melt_3 = ['AEP', 'TCE-AEP']
    melted_df_3 = fdataframe_3.melt(id_vars=columns_to_keep_3, value_vars=columns_to_melt_3, 
                                    var_name='EPType', value_name='Loss')
    melted_df_3.rename(columns={'RPs': 'ReturnPeriod'}, inplace=True)
    final_df_3 = melted_df_3[['EPType', 'Loss', 'ReturnPeriod']]

    columns_to_keep_2 = ['RPs']
    columns_to_melt_2 = ['OEP', 'TCE-OEP']
    melted_df_2 = fdataframe_2.melt(id_vars=columns_to_keep_2, value_vars=columns_to_melt_2, 
                                    var_name='EPType', value_name='Loss')
    melted_df_2.rename(columns={'RPs': 'ReturnPeriod'}, inplace=True)
    final_df_2 = melted_df_2[['EPType', 'Loss', 'ReturnPeriod']]

    final_df_EP_LOB_GU = pd.concat([final_df_2, final_df_3], ignore_index=True)
    new_ep_type_order = ["OEP", "AEP", "TCE-OEP", "TCE-AEP"]
    final_df_EP_LOB_GU['EPType'] = pd.Categorical(final_df_EP_LOB_GU['EPType'], categories=new_ep_type_order, ordered=True)
    final_df_EP_LOB_GU = final_df_EP_LOB_GU.sort_values(by=['EPType', 'ReturnPeriod'], ascending=[True, False]).reset_index(drop=True)

    # Define the schema to match the required Parquet file schema
    schema = pa.schema([
        pa.field('EPType', pa.string(), nullable=True),
        pa.field('Loss', pa.float64(), nullable=True),
        pa.field('ReturnPeriod', pa.float64(), nullable=True),
       
    ])

    # Convert DataFrame to Arrow Table with the specified schema
    table = pa.Table.from_pandas(final_df_EP_LOB_GU, schema=schema)

    # Save to Parquet
    pq.write_table(table, parquet_file_path)

    print(f"Parquet file saved successfully at {parquet_file_path}")

In [63]:
pq_file_path_1=os.path.join(main_folder_path, 'EP', 'Portfolio', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Portfolio_GU_0.parquet')


In [ ]:
try:
    process_and_save_parquet_2(dataframe_1_oep, pq_file_path_1, speriod, samples)
except (NameError, AttributeError):
    pass


In [41]:
dataframe_1_oep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1550197 entries, 0 to 1550196
Data columns (total 4 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   EventId       1550197 non-null  int64         
 1   PeriodId      1550197 non-null  int64         
 2   EventDate     1550197 non-null  datetime64[ns]
 3   Sum_Loss_sum  1550197 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 47.3 MB


In [83]:
dataframe_1_oep.head(10)

,EventId,PeriodId,LobName,Sum_Loss_sum
0,53895970,207557,SPER,2.250859e+10
1,53891598,149148,SPER,2.197889e+10
2,53893301,4312,SPER,2.160561e+10
3,53893301,154312,SPER,2.092983e+10
4,53891598,199148,SPER,1.993543e+10
5,53893301,104312,SPER,1.890958e+10
6,53874490,193067,SPER,1.782878e+10
7,53889489,239080,SPER,1.779150e+10
8,53889489,89080,SPER,1.744444e+10
9,53895970,107557,SPER,1.734298e+10


In [45]:
dataframe_1_oep.to_csv(os.path.join(output_folder_path,'LOB_AUTO.csv'), index=False)

In [39]:
dataframe_1_oep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1550197 entries, 0 to 1550196
Data columns (total 10 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   LobName       1550197 non-null  object        
 1   LobId         1550197 non-null  int64         
 2   EventId       1550197 non-null  int64         
 3   PeriodId      1550197 non-null  int64         
 4   Weight        1550197 non-null  float64       
 5   EventDate     1550197 non-null  datetime64[ns]
 6   LossDate      1550197 non-null  datetime64[ns]
 7   Region        1550197 non-null  object        
 8   Peril         1550197 non-null  object        
 9   Sum_Loss_sum  1550197 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(3), object(3)
memory usage: 118.3+ MB


In [38]:
dataframe_1_oep.head(10)

,LobName,LobId,EventId,PeriodId,Weight,EventDate,LossDate,Region,Peril,Sum_Loss_sum
0,AUTO,2,53891598,149148,0.000004,2020-05-13,2020-05-13,EU,WS,1.654085e+09
1,AUTO,2,53891598,199148,0.000004,2020-05-13,2020-05-13,EU,WS,1.509087e+09
2,AUTO,2,53895970,207557,0.000004,2020-05-30,2020-05-30,EU,WS,1.359457e+09
3,AUTO,2,53893301,4312,0.000004,2020-11-10,2020-11-10,EU,WS,1.319517e+09
4,AUTO,2,53893301,154312,0.000004,2020-11-10,2020-11-10,EU,WS,1.253157e+09
5,AUTO,2,53893301,104312,0.000004,2020-11-10,2020-11-10,EU,WS,1.200635e+09
6,AUTO,2,53874490,193067,0.000004,2020-11-24,2020-11-24,EU,WS,1.184344e+09
7,AUTO,2,53891598,49148,0.000004,2020-05-13,2020-05-13,EU,WS,1.151871e+09
8,AUTO,2,53891598,99148,0.000004,2020-05-13,2020-05-13,EU,WS,1.135624e+09
9,AUTO,2,53889489,239080,0.000004,2020-01-04,2020-01-04,EU,WS,1.104149e+09


In [30]:
processing_folder_path = os.path.join(output_folder_path, 'processing')
os.makedirs(processing_folder_path, exist_ok=True)

# Initialize an empty list to store the results
final_grouped_table_1 = []
final_grouped_table_2 = []

# Process each Parquet file individually
for file in parquet_files:
    # Read the Parquet file into a PyArrow Table
    table = pq.read_table(file)
    
    # Perform the aggregation: max the Loss column grouped by EventId, PeriodId, LobName, and LocationId
    table = table.filter(pc.equal(table['LobName'], 'AUTO'))

    grouped_table_1 = table.group_by(['EventId', 'PeriodId', 'EventDate']).aggregate([('Loss', 'sum')])
    grouped_table_1 = grouped_table_1.rename_columns(['EventId', 'PeriodId', 'EventDate', 'Sum_Loss'])
    #grouped_table_1 = grouped_table_1.group_by(['EventId', 'PeriodId', 'LobName']).aggregate([('Sum_Loss', 'sum')])
   # grouped_table_1 = grouped_table_1.rename_columns(['EventId', 'PeriodId', 'LobName', 'Max_Loss'])
    
    # Perform the aggregation: sum the Loss column grouped by EventId, PeriodId, and LobName
    grouped_table_2 = table.group_by(['EventId', 'PeriodId']).aggregate([('Loss', 'sum')])
    grouped_table_2 = grouped_table_2.rename_columns(['EventId', 'PeriodId','Sum_Loss'])
    
    # Write intermediate results to disk
    pq.write_table(grouped_table_1, os.path.join(processing_folder_path, f'grouped_table_1_{os.path.basename(file)}'))
    pq.write_table(grouped_table_2, os.path.join(processing_folder_path, f'grouped_table_2_{os.path.basename(file)}'))

# Read all intermediate files and concatenate them
intermediate_files_1 = [os.path.join(processing_folder_path, f) for f in os.listdir(processing_folder_path) if f.startswith('grouped_table_1_')]
intermediate_files_2 = [os.path.join(processing_folder_path, f) for f in os.listdir(processing_folder_path) if f.startswith('grouped_table_2_')]

final_grouped_table_1 = [pq.read_table(f) for f in intermediate_files_1]
final_grouped_table_2 = [pq.read_table(f) for f in intermediate_files_2]

final_table_1 = pa.concat_tables(final_grouped_table_1)
final_table_2 = pa.concat_tables(final_grouped_table_2)

# Perform final grouping and sorting
f_grouped_table_1 = final_table_1.group_by(['EventId', 'PeriodId']).aggregate([('Sum_Loss', 'sum')])
f_grouped_table_2 = final_table_2.group_by(['EventId', 'PeriodId']).aggregate([('Sum_Loss', 'sum')])
sorted_final_table_1 = f_grouped_table_1.sort_by([('Sum_Loss_sum', 'descending')])
sorted_final_table_2 = f_grouped_table_2.sort_by([('Sum_Loss_sum', 'descending')])

# Convert to pandas DataFrames
dataframe_1_oep = sorted_final_table_1.to_pandas()
dataframe_1_aep = sorted_final_table_2.to_pandas()

# you can delete the steps below later

# Write the final concatenated files to disk
final_oep_path = os.path.join(output_folder_path, 'final_dataframe_1_oep.parquet')
final_aep_path = os.path.join(output_folder_path, 'final_dataframe_1_aep.parquet')
pq.write_table(sorted_final_table_1, final_oep_path)
pq.write_table(sorted_final_table_2, final_aep_path)

# Delete all non-concatenated files
for f in intermediate_files_1 + intermediate_files_2:
    os.remove(f)

print(f'Final OEP file path: {final_oep_path}')
print(f'Final AEP file path: {final_aep_path}')



ArrowInvalid: No match for FieldRef.Name(EventDate) in EventId: int64 not null
PeriodId: int64 not null
Sum_Loss: double

In [46]:
#test2

In [4]:
flush_cache()
speriod=50000
samples=5
# Define the folder containing the Parquet files
folder_path = r'D:\RISHIN\13_ILC_TASK1\input\PARQUET_FILES'

# List all Parquet files in the folder
parquet_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.parquet')]
output_folder_path = "D:\RISHIN\TESTING FOLDER\ILC_TEST_4"

# Check if there are any Parquet files in the folder
if parquet_files:
    # Read the first Parquet file in chunks
    parquet_file = pq.ParquetFile(parquet_files[0])
    for batch in parquet_file.iter_batches(batch_size=1000):
        # Convert the first batch to a PyArrow Table
        table = pa.Table.from_batches([batch])
        
        # Convert the PyArrow Table to a Pandas DataFrame
        df = table.to_pandas()
        
        # Extract the first value of LocationName and split it by '_'
        location_name = df['LocationName'].iloc[0]
        country = location_name.split('_')[0]
        
        
        # Define the main folder path
        main_folder_path = os.path.join(output_folder_path, f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_Losses')
        
        # Define subfolders
        subfolders = ['EP', 'PLT', 'STATS']
        nested_folders = ['Lob', 'Portfolio']
        innermost_folders = ['GR', 'GU']
        
        # Create the main folder and subfolders
        for subfolder in subfolders:
            subfolder_path = os.path.join(main_folder_path, subfolder)
            os.makedirs(subfolder_path, exist_ok=True)
            
            for nested_folder in nested_folders:
                nested_folder_path = os.path.join(subfolder_path, nested_folder)
                os.makedirs(nested_folder_path, exist_ok=True)
                
                for innermost_folder in innermost_folders:
                    innermost_folder_path = os.path.join(nested_folder_path, innermost_folder)
                    os.makedirs(innermost_folder_path, exist_ok=True)
        
        print(f"Folders created successfully at {main_folder_path}")
        break  # Process only the first batch
else:
    print("No Parquet files found in the specified folder.")

Folders created successfully at D:\RISHIN\TESTING FOLDER\ILC_TEST_4\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses


In [5]:
import pyarrow as pa
import pyarrow.parquet as pq
import os

processing_folder_path = os.path.join(output_folder_path, 'processing')
os.makedirs(processing_folder_path, exist_ok=True)

# Initialize an empty list to store the results
final_grouped_table_1 = []
final_grouped_table_2 = []

# Process each Parquet file individually
for file in parquet_files:
    # Read the Parquet file into a PyArrow Table
    table = pq.read_table(file)
    
    # Perform the aggregation: max the Loss column grouped by EventId, PeriodId, LobName, and LocationId
    grouped_table_1 = table.group_by(['EventId', 'PeriodId', 'LobName', 'EventDate']).aggregate([('Loss', 'sum')])
    grouped_table_1 = grouped_table_1.rename_columns(['EventId', 'PeriodId', 'LobName', 'EventDate', 'Sum_Loss'])
    #grouped_table_1 = grouped_table_1.group_by(['EventId', 'PeriodId', 'LobName']).aggregate([('Sum_Loss', 'sum')])
   # grouped_table_1 = grouped_table_1.rename_columns(['EventId', 'PeriodId', 'LobName', 'Max_Loss'])
    
    # Perform the aggregation: sum the Loss column grouped by EventId, PeriodId, and LobName
    grouped_table_2 = table.group_by(['EventId', 'PeriodId', 'LobName']).aggregate([('Loss', 'sum')])
    grouped_table_2 = grouped_table_2.rename_columns(['EventId', 'PeriodId', 'LobName', 'Sum_Loss'])
    
    # Write intermediate results to disk
    pq.write_table(grouped_table_1, os.path.join(processing_folder_path, f'grouped_table_1_{os.path.basename(file)}'))
    pq.write_table(grouped_table_2, os.path.join(processing_folder_path, f'grouped_table_2_{os.path.basename(file)}'))

# Read all intermediate files and concatenate them
intermediate_files_1 = [os.path.join(processing_folder_path, f) for f in os.listdir(processing_folder_path) if f.startswith('grouped_table_1_')]
intermediate_files_2 = [os.path.join(processing_folder_path, f) for f in os.listdir(processing_folder_path) if f.startswith('grouped_table_2_')]

final_grouped_table_1 = [pq.read_table(f) for f in intermediate_files_1]
final_grouped_table_2 = [pq.read_table(f) for f in intermediate_files_2]

final_table_1 = pa.concat_tables(final_grouped_table_1)
final_table_2 = pa.concat_tables(final_grouped_table_2)

# Perform final grouping and sorting
f_grouped_table_1 = final_table_1.group_by(['EventId', 'PeriodId', 'LobName']).aggregate([('Sum_Loss', 'sum')])
f_grouped_table_2 = final_table_2.group_by(['EventId', 'PeriodId', 'LobName']).aggregate([('Sum_Loss', 'sum')])
sorted_final_table_1 = f_grouped_table_1.sort_by([('Sum_Loss_sum', 'descending')])
sorted_final_table_2 = f_grouped_table_2.sort_by([('Sum_Loss_sum', 'descending')])

# Convert to pandas DataFrames
dataframe_1_oep = sorted_final_table_1.to_pandas()
dataframe_1_aep = sorted_final_table_2.to_pandas()

# you can delete the steps below later

# Write the final concatenated files to disk
final_oep_path = os.path.join(output_folder_path, 'final_dataframe_1_oep.parquet')
final_aep_path = os.path.join(output_folder_path, 'final_dataframe_1_aep.parquet')
pq.write_table(sorted_final_table_1, final_oep_path)
pq.write_table(sorted_final_table_2, final_aep_path)

# Delete all non-concatenated files
for f in intermediate_files_1 + intermediate_files_2:
    os.remove(f)

print(f'Final OEP file path: {final_oep_path}')
print(f'Final AEP file path: {final_aep_path}')

Final OEP file path: D:\RISHIN\TESTING FOLDER\ILC_TEST_4\final_dataframe_1_oep.parquet
Final AEP file path: D:\RISHIN\TESTING FOLDER\ILC_TEST_4\final_dataframe_1_aep.parquet


In [11]:
dataframe_2 = dataframe_1_oep.groupby(['PeriodId'], as_index=False).agg({'Sum_Loss_sum': 'max'})
dataframe_2.rename(columns={'Sum_Loss_sum': 'Max_Loss'}, inplace=True)
dataframe_2 = dataframe_2.sort_values(by='Max_Loss', ascending=False)
dataframe_2.to_csv(r'D:\RISHIN\Rough\dataframe_grouped2.csv', index=False)


In [12]:
dataframe_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 248280 entries, 206130 to 42881
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   PeriodId  248280 non-null  int64  
 1   Max_Loss  248280 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 5.7 MB


In [86]:
# Filter and assign to variables if not empty
if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'AGR'].empty:
    daf_AGR_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'AGR']

if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'AUTO'].empty:
    daf_AUTO_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'AUTO']

if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'COM'].empty:
    daf_COM_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'COM']

if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'IND'].empty:
    daf_IND_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'IND']

if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'SPER'].empty:
    daf_SPER_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'SPER']

if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'FRST'].empty:
    daf_FRST_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'FRST']

if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'GLH'].empty:
    daf_GLH_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'GLH']

if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'AGR'].empty:
    daf_AGR_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'AGR']

if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'AUTO'].empty:
    daf_AUTO_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'AUTO']

if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'COM'].empty:
    daf_COM_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'COM']

if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'IND'].empty:
    daf_IND_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'IND']

if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'SPER'].empty:
    daf_SPER_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'SPER']

if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'FRST'].empty:
    daf_FRST_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'FRST']

if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'GLH'].empty:
    daf_GLH_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'GLH']


import decimal
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

def process_and_save_parquet_2(dataframe_1_oep, dataframe_1_aep, parquet_file_path, speriod, samples):
    # Group and aggregate dataframes
    dataframe_2 = dataframe_1_oep.groupby(['PeriodId', 'LobName'], as_index=False).agg({'Sum_Loss_sum': 'max'})
    dataframe_2.rename(columns={'Sum_Loss_sum': 'Max_Loss'}, inplace=True)
    dataframe_2 = dataframe_2.sort_values(by='Max_Loss', ascending=False)

    dataframe_3 = dataframe_1_aep.groupby(['PeriodId', 'LobName'], as_index=False).agg({'Sum_Loss_sum': 'sum'})
    dataframe_3.rename(columns={'Sum_Loss_sum': 'S_Sum_Loss'}, inplace=True)
    dataframe_3 = dataframe_3.sort_values(by='S_Sum_Loss', ascending=False)

    # Convert columns to decimal
    dataframe_2['Max_Loss'] = dataframe_2['Max_Loss'].apply(lambda x: decimal.Decimal(str(x)))
    dataframe_3['S_Sum_Loss'] = dataframe_3['S_Sum_Loss'].apply(lambda x: decimal.Decimal(str(x)))

    # Calculate rates and cumulative rates using decimal.Decimal
    rate = decimal.Decimal(1) / (decimal.Decimal(speriod) * decimal.Decimal(samples))
    dataframe_2['rate'] = rate.quantize(decimal.Decimal('1.000000'))
    dataframe_2['cumrate'] = dataframe_2['rate'].cumsum().apply(lambda x: decimal.Decimal(str(x)).quantize(decimal.Decimal('1.000000')))
    dataframe_2['RPs'] = dataframe_2['cumrate'].apply(lambda x: decimal.Decimal(1) / x)
    dataframe_2['TCE_OEP_1'] = ((dataframe_2['Max_Loss'] - dataframe_2['Max_Loss'].shift(-1).fillna(0)) * 
                              (dataframe_2['cumrate'] + dataframe_2['cumrate'].shift(-1).fillna(0)) * decimal.Decimal(0.5))
    dataframe_2['TCE_OEP_2'] = (dataframe_2['TCE_OEP_1'].shift().fillna(0).cumsum().apply(lambda x: decimal.Decimal(str(x))) * dataframe_2['RPs'])
    dataframe_2['TCE_OEP_Final'] = (dataframe_2['TCE_OEP_2'] + dataframe_2['Max_Loss'])

    dataframe_3['rate'] = rate
    dataframe_3['cumrate'] = dataframe_3['rate'].cumsum().apply(lambda x: decimal.Decimal(str(x)))
    dataframe_3['RPs'] = dataframe_3['cumrate'].apply(lambda x: decimal.Decimal(1) / x)
    dataframe_3['TCE_AEP_1'] = ((dataframe_3['S_Sum_Loss'] - dataframe_3['S_Sum_Loss'].shift(-1).fillna(0)) * 
                              (dataframe_3['cumrate'] + dataframe_3['cumrate'].shift(-1).fillna(0)) * decimal.Decimal(0.5))
    dataframe_3['TCE_AEP_2'] = (dataframe_3['TCE_AEP_1'].shift().fillna(0).cumsum().apply(lambda x: decimal.Decimal(str(x))) * dataframe_3['RPs'])
    dataframe_3['TCE_AEP_Final'] = (dataframe_3['TCE_AEP_2'] + dataframe_3['S_Sum_Loss'])

    # Filter by RPs values
    rps_values = [10000, 5000, 1000, 500, 250, 200, 100, 50, 25, 10, 5, 2]
    fdataframe_2 = pd.DataFrame()
    fdataframe_3 = pd.DataFrame()

    for value in rps_values:
        closest_index_2 = (dataframe_2['RPs'] - decimal.Decimal(value)).abs().idxmin()
        closest_index_3 = (dataframe_3['RPs'] - decimal.Decimal(value)).abs().idxmin()
        fdataframe_2 = pd.concat([fdataframe_2, dataframe_2.loc[[closest_index_2]]])
        fdataframe_3 = pd.concat([fdataframe_3, dataframe_3.loc[[closest_index_3]]])

    # Rename columns
    fdataframe_3.rename(columns={'S_Sum_Loss': 'AEP', 'TCE_AEP_Final': 'TCE-AEP'}, inplace=True)
    fdataframe_2.rename(columns={'Max_Loss': 'OEP', 'TCE_OEP_Final': 'TCE-OEP'}, inplace=True)

    # Map LobName to LobId
    lobname_to_lobid = {
        'AGR': 1,
        'AUTO': 2,
        'COM': 3,
        'IND': 4,
        'SPER': 5,
        'FRST': 6,
        'GLH': 7
    }

    fdataframe_2['LobId'] = fdataframe_2['LobName'].map(lobname_to_lobid)
    fdataframe_3['LobId'] = fdataframe_3['LobName'].map(lobname_to_lobid)

    # Cast LobId to Decimal with precision 38 and scale 0
    fdataframe_2['LobId'] = fdataframe_2['LobId'].apply(lambda x: decimal.Decimal(x).scaleb(-0))
    fdataframe_3['LobId'] = fdataframe_3['LobId'].apply(lambda x: decimal.Decimal(x).scaleb(-0))

    # Melt dataframes
    columns_to_keep_3 = ['RPs', 'LobId', 'LobName']
    columns_to_melt_3 = ['AEP', 'TCE-AEP']
    melted_df_3 = fdataframe_3.melt(id_vars=columns_to_keep_3, value_vars=columns_to_melt_3, 
                                    var_name='EPType', value_name='Loss')
    melted_df_3.rename(columns={'RPs': 'ReturnPeriod'}, inplace=True)
    final_df_3 = melted_df_3[['EPType', 'Loss', 'ReturnPeriod', 'LobId', 'LobName']]

    columns_to_keep_2 = ['RPs', 'LobId', 'LobName']
    columns_to_melt_2 = ['OEP', 'TCE-OEP']
    melted_df_2 = fdataframe_2.melt(id_vars=columns_to_keep_2, value_vars=columns_to_melt_2, 
                                    var_name='EPType', value_name='Loss')
    melted_df_2.rename(columns={'RPs': 'ReturnPeriod'}, inplace=True)
    final_df_2 = melted_df_2[['EPType', 'Loss', 'ReturnPeriod', 'LobId', 'LobName']]

    # Concatenate final dataframes
    final_df_EP_LOB_GU = pd.concat([final_df_2, final_df_3], ignore_index=True)
    new_ep_type_order = ["OEP", "AEP", "TCE-OEP", "TCE-AEP"]
    final_df_EP_LOB_GU['EPType'] = pd.Categorical(final_df_EP_LOB_GU['EPType'], categories=new_ep_type_order, ordered=True)
    final_df_EP_LOB_GU = final_df_EP_LOB_GU.sort_values(by=['EPType','Loss'], ascending=[True, False])#.reset_index(drop=True)

    # Define the schema to match the required Parquet file schema
    schema = pa.schema([
        pa.field('EPType', pa.string(), nullable=True),
        pa.field('Loss', pa.decimal128(38, 18), nullable=True),
        pa.field('ReturnPeriod', pa.decimal128(38, 18), nullable=True),
        pa.field('LobId', pa.decimal128(38, 0), nullable=True),
        pa.field('LobName', pa.string(), nullable=True)
    ])

    # Convert DataFrame to Arrow Table with the specified schema
    table = pa.Table.from_pandas(final_df_EP_LOB_GU, schema=schema)

    # Save to Parquet
    pq.write_table(table, parquet_file_path)

    print(f"Parquet file saved successfully at {parquet_file_path}")

# Example usage
# process_and_save_parquet_2(dataframe_1_oep, dataframe_1_aep, 'output.parquet', 100, 1000)
    
pq_file_path_1=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_0.parquet')

pq_file_path_2=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_1.parquet')

pq_file_path_3=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_2.parquet')

pq_file_path_4=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_3.parquet')

pq_file_path_5=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_4.parquet')

pq_file_path_6=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_5.parquet')

pq_file_path_7=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_6.parquet')


try:
    process_and_save_parquet_2(daf_AGR_oep, daf_AGR_aep, pq_file_path_1, speriod, samples)
except (NameError, AttributeError):
    pass

try:
    process_and_save_parquet_2(daf_AUTO_oep, daf_AUTO_aep, pq_file_path_2, speriod, samples)
except (NameError, AttributeError):
    pass

try:
    process_and_save_parquet_2(daf_COM_oep, daf_COM_aep, pq_file_path_3, speriod, samples)
except (NameError, AttributeError):
    pass

try:
    process_and_save_parquet_2(daf_IND_oep, daf_IND_aep, pq_file_path_4, speriod, samples)
except (NameError, AttributeError):
    pass

try:
    process_and_save_parquet_2(daf_SPER_oep, daf_SPER_aep, pq_file_path_5, speriod, samples)
except (NameError, AttributeError):
    pass

try:
    process_and_save_parquet_2(daf_FRST_oep, daf_FRST_aep, pq_file_path_6, speriod, samples)
except (NameError, AttributeError):
    pass

try:
    process_and_save_parquet_2(daf_GLH_oep, daf_GLH_aep, pq_file_path_7, speriod, samples)
except (NameError, AttributeError):
    pass


Parquet file saved successfully at D:\RISHIN\TESTING FOLDER\ILC_TEST_4\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\EP\Lob\GU\ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_0.parquet
Parquet file saved successfully at D:\RISHIN\TESTING FOLDER\ILC_TEST_4\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\EP\Lob\GU\ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_1.parquet
Parquet file saved successfully at D:\RISHIN\TESTING FOLDER\ILC_TEST_4\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\EP\Lob\GU\ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_2.parquet
Parquet file saved successfully at D:\RISHIN\TESTING FOLDER\ILC_TEST_4\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\EP\Lob\GU\ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_3.parquet
Parquet file saved successfully at D:\RISHIN\TESTING FOLDER\ILC_TEST_4\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\EP\Lob\GU\ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_4.parquet


In [92]:
# import pyarrow as pa
# import pyarrow.parquet as pq
# import os

# processing_folder_path = os.path.join(output_folder_path, 'processing')
# os.makedirs(processing_folder_path, exist_ok=True)

# # Initialize an empty list to store the results
# final_grouped_table_1 = []
# final_grouped_table_2 = []

# # Process each Parquet file individually
# for file in parquet_files:
#     # Read the Parquet file into a PyArrow Table
#     table = pq.read_table(file)
    
#     # Perform the aggregation: max the Loss column grouped by EventId, PeriodId, LobName, and LocationId
#     grouped_table_1 = table.group_by(['EventId', 'PeriodId','Loss' , 'LobName', 'EventDate']).aggregate([('Loss', 'sum')])
#     grouped_table_1 = grouped_table_1.rename_columns(['EventId', 'PeriodId','Loss', 'LobName', 'EventDate', 'Sum_Loss'])
#     #grouped_table_1 = grouped_table_1.group_by(['EventId', 'PeriodId', 'LobName']).aggregate([('Sum_Loss', 'sum')])
#     #grouped_table_1 = grouped_table_1.rename_columns(['EventId', 'PeriodId', 'LobName', 'Max_Loss'])
    
#     # Perform the aggregation: sum the Loss column grouped by EventId, PeriodId, and LobName
#     grouped_table_2= table.group_by(['EventId', 'PeriodId','Loss','LobName']).aggregate([('Loss', 'sum')])
#     grouped_table_2 = grouped_table_2.rename_columns(['EventId', 'PeriodId','Loss', 'LobName', 'Sum_Loss'])
    
#     # Write intermediate results to disk
#     pq.write_table(grouped_table_1, os.path.join(processing_folder_path, f'grouped_table_1_{os.path.basename(file)}'))
#     pq.write_table(grouped_table_2, os.path.join(processing_folder_path, f'grouped_table_2_{os.path.basename(file)}'))

# # Read all intermediate files and concatenate them
# intermediate_files_1 = [os.path.join(processing_folder_path, f) for f in os.listdir(processing_folder_path) if f.startswith('grouped_table_1_')]
# intermediate_files_2 = [os.path.join(processing_folder_path, f) for f in os.listdir(processing_folder_path) if f.startswith('grouped_table_2_')]

# final_grouped_table_1 = [pq.read_table(f) for f in intermediate_files_1]
# final_grouped_table_2 = [pq.read_table(f) for f in intermediate_files_2]

# final_table_1 = pa.concat_tables(final_grouped_table_1)
# final_table_2 = pa.concat_tables(final_grouped_table_2)

# # Perform final grouping and sorting
# f_grouped_table_1 = final_table_1.group_by(['EventId', 'PeriodId', 'LobName','Loss']).aggregate([('Sum_Loss', 'sum')])
# f_grouped_table_2 = final_table_2.group_by(['EventId', 'PeriodId', 'LobName','Loss']).aggregate([('Sum_Loss', 'sum')])
# sorted_final_table_1 = f_grouped_table_1.sort_by([('Loss', 'descending')])
# sorted_final_table_2 = f_grouped_table_2.sort_by([('Loss', 'descending')])

# # Convert to pandas DataFrames
# dataframe_1_oep = sorted_final_table_1.to_pandas()
# dataframe_1_aep = sorted_final_table_2.to_pandas()

# # you can delete the steps below later

# # Write the final concatenated files to disk
# final_oep_path = os.path.join(output_folder_path, 'final_dataframe_1_oep.parquet')
# final_aep_path = os.path.join(output_folder_path, 'final_dataframe_1_aep.parquet')
# pq.write_table(sorted_final_table_1, final_oep_path)
# pq.write_table(sorted_final_table_2, final_aep_path)

# # Delete all non-concatenated files
# for f in intermediate_files_1 + intermediate_files_2:
#     os.remove(f)

# print(f'Final OEP file path: {final_oep_path}')
# print(f'Final AEP file path: {final_aep_path}')

ArrowMemoryError: malloc of size 8388608 failed

In [88]:
# # Filter and assign to variables if not empty
# if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'AGR'].empty:
#     daf_AGR_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'AGR']

# if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'AUTO'].empty:
#     daf_AUTO_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'AUTO']

# if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'COM'].empty:
#     daf_COM_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'COM']

# if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'IND'].empty:
#     daf_IND_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'IND']

# if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'SPER'].empty:
#     daf_SPER_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'SPER']

# if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'FRST'].empty:
#     daf_FRST_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'FRST']

# if not dataframe_1_oep[dataframe_1_oep['LobName'] == 'GLH'].empty:
#     daf_GLH_oep = dataframe_1_oep[dataframe_1_oep['LobName'] == 'GLH']

# if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'AGR'].empty:
#     daf_AGR_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'AGR']

# if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'AUTO'].empty:
#     daf_AUTO_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'AUTO']

# if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'COM'].empty:
#     daf_COM_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'COM']

# if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'IND'].empty:
#     daf_IND_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'IND']

# if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'SPER'].empty:
#     daf_SPER_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'SPER']

# if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'FRST'].empty:
#     daf_FRST_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'FRST']

# if not dataframe_1_aep[dataframe_1_aep['LobName'] == 'GLH'].empty:
#     daf_GLH_aep = dataframe_1_aep[dataframe_1_aep['LobName'] == 'GLH']


# import decimal
# import pandas as pd
# import pyarrow as pa
# import pyarrow.parquet as pq

# def process_and_save_parquet_2(dataframe_1_oep, dataframe_1_aep, parquet_file_path, speriod, samples):
#     # Group and aggregate dataframes
#     dataframe_2 = dataframe_1_oep.groupby(['PeriodId', 'LobName','Loss'], as_index=False).agg({'Sum_Loss_sum': 'max'})
#     dataframe_2.rename(columns={'Sum_Loss_sum': 'Max_Loss'}, inplace=True)
#     dataframe_2 = dataframe_2.sort_values(by='Max_Loss', ascending=False)

#     dataframe_3 = dataframe_1_aep.groupby(['PeriodId', 'LobName','Loss'], as_index=False).agg({'Sum_Loss_sum': 'sum'})
#     dataframe_3.rename(columns={'Sum_Loss_sum': 'S_Sum_Loss'}, inplace=True)
#     dataframe_3 = dataframe_3.sort_values(by='S_Sum_Loss', ascending=False)

#     # Convert columns to decimal
#     dataframe_2['Max_Loss'] = dataframe_2['Max_Loss'].apply(lambda x: decimal.Decimal(str(x)))
#     dataframe_3['S_Sum_Loss'] = dataframe_3['S_Sum_Loss'].apply(lambda x: decimal.Decimal(str(x)))

#     # Calculate rates and cumulative rates using decimal.Decimal
#     rate = decimal.Decimal(1) / (decimal.Decimal(speriod) * decimal.Decimal(samples))
#     dataframe_2['rate'] = rate.quantize(decimal.Decimal('1.000000'))
#     dataframe_2['cumrate'] = dataframe_2['rate'].cumsum().apply(lambda x: decimal.Decimal(str(x)).quantize(decimal.Decimal('1.000000')))
#     dataframe_2['RPs'] = dataframe_2['cumrate'].apply(lambda x: decimal.Decimal(1) / x)
#     dataframe_2['TCE_OEP_1'] = ((dataframe_2['Max_Loss'] - dataframe_2['Max_Loss'].shift(-1).fillna(0)) * 
#                               (dataframe_2['cumrate'] + dataframe_2['cumrate'].shift(-1).fillna(0)) * decimal.Decimal(0.5))
#     dataframe_2['TCE_OEP_2'] = (dataframe_2['TCE_OEP_1'].shift().fillna(0).cumsum().apply(lambda x: decimal.Decimal(str(x))) * dataframe_2['RPs'])
#     dataframe_2['TCE_OEP_Final'] = (dataframe_2['TCE_OEP_2'] + dataframe_2['Max_Loss'])

#     dataframe_3['rate'] = rate
#     dataframe_3['cumrate'] = dataframe_3['rate'].cumsum().apply(lambda x: decimal.Decimal(str(x)))
#     dataframe_3['RPs'] = dataframe_3['cumrate'].apply(lambda x: decimal.Decimal(1) / x)
#     dataframe_3['TCE_AEP_1'] = ((dataframe_3['S_Sum_Loss'] - dataframe_3['S_Sum_Loss'].shift(-1).fillna(0)) * 
#                               (dataframe_3['cumrate'] + dataframe_3['cumrate'].shift(-1).fillna(0)) * decimal.Decimal(0.5))
#     dataframe_3['TCE_AEP_2'] = (dataframe_3['TCE_AEP_1'].shift().fillna(0).cumsum().apply(lambda x: decimal.Decimal(str(x))) * dataframe_3['RPs'])
#     dataframe_3['TCE_AEP_Final'] = (dataframe_3['TCE_AEP_2'] + dataframe_3['S_Sum_Loss'])

#     # Filter by RPs values
#     rps_values = [10000, 5000, 1000, 500, 250, 200, 100, 50, 25, 10, 5, 2]
#     fdataframe_2 = pd.DataFrame()
#     fdataframe_3 = pd.DataFrame()

#     for value in rps_values:
#         closest_index_2 = (dataframe_2['RPs'] - decimal.Decimal(value)).abs().idxmin()
#         closest_index_3 = (dataframe_3['RPs'] - decimal.Decimal(value)).abs().idxmin()
#         fdataframe_2 = pd.concat([fdataframe_2, dataframe_2.loc[[closest_index_2]]])
#         fdataframe_3 = pd.concat([fdataframe_3, dataframe_3.loc[[closest_index_3]]])

#     # Rename columns
#     fdataframe_3.rename(columns={'S_Sum_Loss': 'AEP', 'TCE_AEP_Final': 'TCE-AEP'}, inplace=True)
#     fdataframe_2.rename(columns={'Max_Loss': 'OEP', 'TCE_OEP_Final': 'TCE-OEP'}, inplace=True)

#     # Map LobName to LobId
#     lobname_to_lobid = {
#         'AGR': 1,
#         'AUTO': 2,
#         'COM': 3,
#         'IND': 4,
#         'SPER': 5,
#         'FRST': 6,
#         'GLH': 7
#     }

#     fdataframe_2['LobId'] = fdataframe_2['LobName'].map(lobname_to_lobid)
#     fdataframe_3['LobId'] = fdataframe_3['LobName'].map(lobname_to_lobid)

#     # Cast LobId to Decimal with precision 38 and scale 0
#     fdataframe_2['LobId'] = fdataframe_2['LobId'].apply(lambda x: decimal.Decimal(x).scaleb(-0))
#     fdataframe_3['LobId'] = fdataframe_3['LobId'].apply(lambda x: decimal.Decimal(x).scaleb(-0))

#     # Melt dataframes
#     columns_to_keep_3 = ['RPs', 'LobId', 'LobName']
#     columns_to_melt_3 = ['AEP', 'TCE-AEP']
#     melted_df_3 = fdataframe_3.melt(id_vars=columns_to_keep_3, value_vars=columns_to_melt_3, 
#                                     var_name='EPType', value_name='Loss_c')
#     melted_df_3.rename(columns={'RPs': 'ReturnPeriod'}, inplace=True)
#     final_df_3 = melted_df_3[['EPType', 'Loss','Loss_c','ReturnPeriod', 'LobId', 'LobName']]

#     columns_to_keep_2 = ['RPs', 'LobId', 'LobName']
#     columns_to_melt_2 = ['OEP', 'TCE-OEP']
#     melted_df_2 = fdataframe_2.melt(id_vars=columns_to_keep_2, value_vars=columns_to_melt_2, 
#                                     var_name='EPType', value_name='Loss')
#     melted_df_2.rename(columns={'RPs': 'ReturnPeriod'}, inplace=True)
#     final_df_2 = melted_df_2[['EPType', 'Loss', 'Loss_c','ReturnPeriod', 'LobId', 'LobName']]

#     # Concatenate final dataframes
#     final_df_EP_LOB_GU = pd.concat([final_df_2, final_df_3], ignore_index=True)
#     new_ep_type_order = ["OEP", "AEP", "TCE-OEP", "TCE-AEP"]
#     final_df_EP_LOB_GU['EPType'] = pd.Categorical(final_df_EP_LOB_GU['EPType'], categories=new_ep_type_order, ordered=True)
#     final_df_EP_LOB_GU = final_df_EP_LOB_GU.sort_values(by=['EPType','Loss'], ascending=[True, False])#.reset_index(drop=True)

#     # Define the schema to match the required Parquet file schema
#     schema = pa.schema([
#         pa.field('EPType', pa.string(), nullable=True),
#         pa.field('Loss', pa.decimal128(38, 18), nullable=True),
#         pa.field('Loss_c', pa.decimal128(38, 18), nullable=True),
#         pa.field('ReturnPeriod', pa.decimal128(38, 18), nullable=True),
#         pa.field('LobId', pa.decimal128(38, 0), nullable=True),
#         pa.field('LobName', pa.string(), nullable=True)
#     ])

#     # Convert DataFrame to Arrow Table with the specified schema
#     table = pa.Table.from_pandas(final_df_EP_LOB_GU, schema=schema)

#     # Save to Parquet
#     pq.write_table(table, parquet_file_path)

#     print(f"Parquet file saved successfully at {parquet_file_path}")

# # Example usage
# # process_and_save_parquet_2(dataframe_1_oep, dataframe_1_aep, 'output.parquet', 100, 1000)
    
# pq_file_path_1=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_0.parquet')

# pq_file_path_2=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_1.parquet')

# pq_file_path_3=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_2.parquet')

# pq_file_path_4=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_3.parquet')

# pq_file_path_5=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_4.parquet')

# pq_file_path_6=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_5.parquet')

# pq_file_path_7=os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_6.parquet')


# try:
#     process_and_save_parquet_2(daf_AGR_oep, daf_AGR_aep, pq_file_path_1, speriod, samples)
# except (NameError, AttributeError):
#     pass

# try:
#     process_and_save_parquet_2(daf_AUTO_oep, daf_AUTO_aep, pq_file_path_2, speriod, samples)
# except (NameError, AttributeError):
#     pass

# try:
#     process_and_save_parquet_2(daf_COM_oep, daf_COM_aep, pq_file_path_3, speriod, samples)
# except (NameError, AttributeError):
#     pass

# try:
#     process_and_save_parquet_2(daf_IND_oep, daf_IND_aep, pq_file_path_4, speriod, samples)
# except (NameError, AttributeError):
#     pass

# try:
#     process_and_save_parquet_2(daf_SPER_oep, daf_SPER_aep, pq_file_path_5, speriod, samples)
# except (NameError, AttributeError):
#     pass

# try:
#     process_and_save_parquet_2(daf_FRST_oep, daf_FRST_aep, pq_file_path_6, speriod, samples)
# except (NameError, AttributeError):
#     pass

# try:
#     process_and_save_parquet_2(daf_GLH_oep, daf_GLH_aep, pq_file_path_7, speriod, samples)
# except (NameError, AttributeError):
#     pass


KeyboardInterrupt: 